Reshaping with Hierarchical Indexing
Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

stack: This “rotates” or pivots from the columns in the data to the rows.
unstack: This pivots from the rows into the columns.
I'll illustrate these operations through a series of examples. Consider a small DataFrame with string arrays as row and column indexes:

In [ ]:
#### Code
import pandas as pd
import numpy as np

# Create a DataFrame
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"], name="number"))

# Display the DataFrame
print(data)


number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


Using the stack method on this data pivots the columns into the rows, producing a Series:

In [ ]:
#### Code
# Stack the DataFrame
result = data.stack()

# Display the stacked result
print(result)


state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64


From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with unstack:

In [ ]:
# Unstack the Series back into a DataFrame
unstacked_result = result.unstack()

# Display the unstacked result
print(unstacked_result)


number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


By default, the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:

In [ ]:
# Unstack by level number
unstacked_level0 = result.unstack(level=0)
print(unstacked_level0)

# Unstack by level name
unstacked_level_state = result.unstack(level="state")
print(unstacked_level_state)


Unstacking might introduce missing data if all of the values in the level aren’t found in each subgroup:

In [ ]:
# Create Series
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")

# Concatenate Series
data2 = pd.concat([s1, s2], keys=["one", "two"])

# Display the concatenated Series
print(data2)


one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64


Stacking filters out missing data by default, so the operation is more easily invertible:

In [ ]:
# Unstack the concatenated Series
unstacked_data2 = data2.unstack()

# Display the unstacked result
print(unstacked_data2)

# Stack the unstacked Series
restacked_data2 = unstacked_data2.stack()
print(restacked_data2)

# Stack without dropping NA values
restacked_data2_dropna_false = unstacked_data2.stack(dropna=False)
print(restacked_data2_dropna_false)


        a     b  c  d     e
one     0     1  2  3  <NA>
two  <NA>  <NA>  4  5     6
one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64
one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64


When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:

In [ ]:
# Create a DataFrame with hierarchical indexing
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))

# Display the DataFrame
print(df)

# Unstack by level "state"
unstacked_df = df.unstack(level="state")
print(unstacked_df)


side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10
side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10


As with unstack, when calling stack we can indicate the name of the axis to stack:

In [ ]:
# Stack by level "side"
stacked_side = unstacked_df.stack(level="side")
print(stacked_side)


state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7


Pivoting “Long” to “Wide” Format
A common way to store multiple time series in databases and CSV files is what is sometimes called long or stacked format. In this format, individual values are represented by a single row in a table rather than multiple values per row.

Let’s load some example data and do a small amount of time series wrangling and other data cleaning:

In [ ]:
# Load data
data = pd.read_csv("https://raw.githubusercontent.com/wesm/pydata-book/3rd-edition/examples/macrodata.csv")

# Select a subset of columns
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

# Display the first few rows
print(data.head())


   year  quarter   realgdp  infl  unemp
0  1959        1  2710.349  0.00    5.8
1  1959        2  2778.801  2.34    5.1
2  1959        3  2775.488  2.74    5.3
3  1959        4  2785.204  0.27    5.6
4  1960        1  2847.699  2.31    5.2


First, I use pandas.PeriodIndex (which represents time intervals rather than points in time), discussed in more detail in Chapter 11, to combine the year and quarter columns to set the index to consist of datetime values at the end of each quarter:

In [ ]:
# Load data
data = pd.read_csv("https://raw.githubusercontent.com/wesm/pydata-book/3rd-edition/examples/macrodata.csv")

# Select a subset of columns
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

# Handle potential missing values in 'year' and 'quarter' columns
data['year'] = data['year'].fillna(0).astype(int)  # Replace missing years with 0 and convert to integer
data['quarter'] = data['quarter'].fillna(0).astype(int)  # Replace missing quarters with 0 and convert to integer

# Create a PeriodIndex
periods = pd.PeriodIndex(year=data['year'],
                         quarter=data['quarter'],
                         name="date")

# Assign the PeriodIndex to the DataFrame index (if needed)
# data.index = periods.to_timestamp("D")

# Display the first few rows
print(data.head())
periods

   year  quarter   realgdp  infl  unemp
0  1959        1  2710.349  0.00    5.8
1  1959        2  2778.801  2.34    5.1
2  1959        3  2775.488  2.74    5.3
3  1959        4  2785.204  0.27    5.6
4  1960        1  2847.699  2.31    5.2


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

Here I used the pop method on the DataFrame, which returns a column while deleting it from the DataFrame at the same time.

Then, I select a subset of columns and give the columns index the name "item":

In [ ]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


Here I used the pop method on the DataFrame, which returns a column while deleting it from the DataFrame at the same time.

Then, I select a subset of columns and give the columns index the name "item":

In [ ]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"

In [ ]:
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


Lastly, I reshape with stack, turn the new index levels into columns with reset_index, and finally give the column containing the data values the name "value":

In [ ]:
long_data = (data.stack()
            .reset_index()
            .rename(columns={0: "value"}))

Now, ldata looks like:

In [ ]:
long_data[:10]

,level_0,item,value
0,0,realgdp,2710.349
1,0,infl,0.000
2,0,unemp,5.800
3,1,realgdp,2778.801
4,1,infl,2.340
5,1,unemp,5.100
6,2,realgdp,2775.488
7,2,infl,2.740
8,2,unemp,5.300
9,3,realgdp,2785.204


In this so-called long format for multiple time series, each row in the table represents a single observation.

Data is frequently stored this way in relational SQL databases, as a fixed schema (column names and data types) allows the number of distinct values in the item column to change as data is added to the table. In the previous example, date and item would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins. In some cases, the data may be more difficult to work with in this format; you might prefer to have a DataFrame containing one column per distinct item value indexed by timestamps in the date column. DataFrame’s pivot method performs exactly this transformation:

In [ ]:
long_data = (data.stack()
            .reset_index()
            .rename(columns={0: "value"}))

# Check if 'date' column exists and rename it if necessary
if 'date' not in long_data.columns:
    # Assuming the date information is in one of the index levels
    long_data = long_data.rename(columns={'level_0': 'date'}) # Adjust 'level_0' if the date is in a different level

pivoted = long_data.pivot(index="date", columns="item", values="value")

In [ ]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
0,0.00,2710.349,5.8
1,2.34,2778.801,5.1
2,2.74,2775.488,5.3
3,0.27,2785.204,5.6
4,2.31,2847.699,5.2


The first two values passed are the columns to be used, respectively, as the row and column index, then finally an optional value column to fill the DataFrame. Suppose you had two value columns that you wanted to reshape simultaneously:

In [ ]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,0,realgdp,2710.349,-0.158778
1,0,infl,0.000,-0.076942
2,0,unemp,5.800,1.359143
3,1,realgdp,2778.801,-0.639733
4,1,infl,2.340,-2.150261
5,1,unemp,5.100,1.600584
6,2,realgdp,2775.488,-0.485903
7,2,infl,2.740,-1.110899
8,2,unemp,5.300,1.452980
9,3,realgdp,2785.204,-0.337022


By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [ ]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item  infl   realgdp unemp      infl   realgdp     unemp
date                                                    
0     0.00  2710.349   5.8 -0.076942 -0.158778  1.359143
1     2.34  2778.801   5.1 -2.150261 -0.639733  1.600584
2     2.74  2775.488   5.3 -1.110899 -0.485903  1.452980
3     0.27  2785.204   5.6  0.163513 -0.337022  3.181721
4     2.31  2847.699   5.2 -0.968884 -0.912688  0.066612

In [ ]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
0,0.00,2710.349,5.8
1,2.34,2778.801,5.1
2,2.74,2775.488,5.3
3,0.27,2785.204,5.6
4,2.31,2847.699,5.2


Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack:

In [ ]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item  infl   realgdp unemp      infl   realgdp     unemp
date                                                    
0     0.00  2710.349   5.8 -0.076942 -0.158778  1.359143
1     2.34  2778.801   5.1 -2.150261 -0.639733  1.600584
2     2.74  2775.488   5.3 -1.110899 -0.485903  1.452980
3     0.27  2785.204   5.6  0.163513 -0.337022  3.181721
4     2.31  2847.699   5.2 -0.968884 -0.912688  0.066612

Pivoting “Wide” to “Long” Format
An inverse operation to pivot for DataFrames is pandas.melt. Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input. Let’s look at an example:

In [ ]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                  "A": [1, 2, 3],
                  "B": [4, 5, 6],
                  "C": [7, 8, 9]})

In [ ]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The "key" column may be a group indicator, and the other columns are data values. When using pandas.melt, we must indicate which columns (if any) are group indicators. Let’s use "key" as the only group indicator here:

In [ ]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Using pivot, we can reshape back to the original layout:

In [36]:
reshaped = melted.pivot(index="key", columns="variable",
                       values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Since the result of pivot creates an index from the column used as the row labels, we may want to use reset_index to move the data back into a column:

In [37]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


You can also specify a subset of columns to use as value columns:

In [ ]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

pandas.melt can be used without any group identifiers, too:

In [39]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [38]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
